In [1]:
import pandas
from pandas import read_excel, date_range, DatetimeIndex, DataFrame, to_numeric, concat, to_datetime
from pandas.plotting import lag_plot, autocorrelation_plot
from pandas.tseries.offsets import DateOffset

import numpy
from numpy import logical_not

import statistics
import dateutil.relativedelta
from dateutil.relativedelta import relativedelta

import fbprophet
from fbprophet import Prophet

import calendar
from calendar import monthrange

from datetime import datetime, timedelta
from matplotlib import pyplot

from sklearn.metrics import mean_absolute_error, mean_squared_error

from math import sqrt

Importing plotly failed. Interactive plots will not work.


In [2]:
# cambiamos el valor de estos 4 parametros para que nos muestre dataframes sin truncarlos
pandas.set_option('display.max_rows', 12)
# pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

In [3]:
# Load data using read_excel
transacciones_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')
transacciones_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría,Nivel categoría,iD categoría padre,Nombre categoría padre,Proveedor,Marca,Sector,Financiero,Transferencia,Unnamed: 12,Unnamed: 13,ID,BALANCE,BALANCE_DATE
0,2020-07-24,-1.20,84.0,parking_84,2.0,5.0,vehiculos_5,NaN,NaN,NaN,False,False,NaN,NaN,249236.0,1478.98,2020-07-26
1,2020-07-23,-4.50,117.0,regalos_celebraciones_117,2.0,8.0,gastos_personales_8,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
2,2020-07-23,-13.53,70.0,supermercados_70,2.0,4.0,alimentacion_4,SPAR,SPAR,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
3,2020-07-22,-51.40,222.0,adsl_222,3.0,521.0,Comunicaciones y TV,Jazz Telecom,Jazztel,Compañías telecomunicaciones,False,False,NaN,NaN,NaN,NaN,NaT
4,2020-07-22,-11.00,70.0,supermercados_70,2.0,4.0,alimentacion_4,Mercadona,Mercadona,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,2016-09-05,1533.02,18.0,retribucion_liquida_18,3.0,576.0,Nómina,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
1483,2016-09-05,-617.47,126.0,liquidacion_tarjeta_126,3.0,478.0,tarjetas_credito_478,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaT
1484,2016-09-05,-138.08,125.0,cuotas_125,3.0,127.0,otros_pasivos_127,NaN,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaT
1485,2016-09-01,-32.22,90.0,seguro_salud_90,2.0,6.0,salud_6,Sanitas S.A. de Seguros,Sanitas,Seguros de salud,True,False,NaN,NaN,NaN,NaN,NaT


In [4]:
# nos quedamos con categoría desde 'Fecha transacción' hasta 'Nombre Categoría'
transacciones_df=transacciones_df.iloc[:, 0:4]
transacciones_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría
0,2020-07-24,-1.20,84.0,parking_84
1,2020-07-23,-4.50,117.0,regalos_celebraciones_117
2,2020-07-23,-13.53,70.0,supermercados_70
3,2020-07-22,-51.40,222.0,adsl_222
4,2020-07-22,-11.00,70.0,supermercados_70
...,...,...,...,...
1482,2016-09-05,1533.02,18.0,retribucion_liquida_18
1483,2016-09-05,-617.47,126.0,liquidacion_tarjeta_126
1484,2016-09-05,-138.08,125.0,cuotas_125
1485,2016-09-01,-32.22,90.0,seguro_salud_90


In [5]:
# nos quedamos con las transacciones de la categoría 'impuestos_15'
transacciones_15_df = transacciones_df[transacciones_df['ID Categoría'] == 15.0]
del transacciones_df
transacciones_15_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría
1135,2017-06-14,-519.76,15.0,impuestos_15
1136,2017-06-14,-519.77,15.0,impuestos_15
1137,2017-06-14,-519.76,15.0,impuestos_15
1152,2017-06-05,-519.38,15.0,impuestos_15


In [ ]:
# este nos es un buena categoria pues solo tiene 4 movimientos, 1 el 5-jun 2017 y 3 el 14-jun 2017